In [ ]:
#!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

In [53]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

import os
import csv
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.docstore.document import Document
from langchain_core.document_loaders.base import BaseLoader

try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [54]:
# ChatGPT and Neo4J access data
os.environ["OPENAI_API_KEY"] = "sk-"
os.environ["NEO4J_URI"] = "neo4j+s://4ce4d1e3.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "TJBTGgX8-66Q48YGoVMJYpWZJIT-L13r9JOk-w9xxU0"
os.environ["AURA_INSTANCEID"] = "4ce4d1e3"
os.environ["AURA_INSTANCENAME"] = "Instance01"


graph = Neo4jGraph()

In [55]:
# loading of TravelLine service responses from the text field of csv files in the data folder
class CustomCSVLoader:
    def __init__(self, directory_path: str, text_col_name: str, **kwargs):
        self.directory_path = directory_path
        self.text_col_name = text_col_name

    def load(self) -> list[Document]:
        documents = []
        for file_name in os.listdir(self.directory_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(self.directory_path, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    reader = csv.DictReader(file, delimiter="\t")
                    for row in reader:
                        text = row.get(self.text_col_name, '')
                        if text:
                            #print (text)
                            document = Document(page_content=text, metadata={'file_name': file_name})
                            documents.append(document)
        return documents

directory_path = "data"
text_col_name = "text"
loader = CustomCSVLoader(directory_path, text_col_name)
raw_documents = loader.load()
print (len(raw_documents))
#print (raw_documents[0])


946
page_content='<br>Как закрыть номер на ремонт в расширенном модуле обслуживания<br>=============================================================<br><br><br>Как закрыть номер на ремонт<br>---------------------------<br><br><br>1. В разделе «Управление номерами» → «Обслуживание номеров» нажмите на кнопку «Назначить ремонт»:<br><br><br>![8df1fcd8e2e76c25ab1323875825d2d1.png](https://138018.selcdn.ru/KB_images/omnideskru/13074/281730/8df1fcd8e2e76c25ab1323875825d2d1.png)<br>\xa0<br><br><br>Чтобы назначить ремонт в списке номеров, нажмите на кнопку «Действие» и выберите «Назначить ремонт»:<br><br><br>![7bcefafba009e72f0a81a509ae301396.png](https://138018.selcdn.ru/KB_images/omnideskru/13074/281730/7bcefafba009e72f0a81a509ae301396.png)<br>\xa0<br><br><br>2. Выберите нужный номер, укажите название работ, краткое описание, дату, время начала и окончания ремонта. Нажмите «Применить»:<br><br><br>![3b02458e2b83f8e020014b7afda25712.png](https://138018.selcdn.ru/KB_images/omnideskru/13074/28173

In [57]:
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
#documents = text_splitter.split_documents(raw_documents[:3])
documents = text_splitter.split_documents(raw_documents[:300])
print(len(documents))

1811


In [58]:
# make the graph
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo",openai_api_base="https://api.vsegpt.ru:6070/v1") # gpt-4-0125-preview occasionally has issues
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

Failed to write data to connection IPv4Address(('4ce4d1e3.databases.neo4j.io', 7687)) (IPv4Address(('35.205.213.74', 7687)))
Failed to write data to connection ResolvedIPv4Address(('35.205.213.74', 7687)) (IPv4Address(('35.205.213.74', 7687)))


BadRequestError: Error code: 400 - {'error': {'message': 'Potentially out of budget: 4808->700, expected price 0.5181, but you have only -0.059480 on account. Please, add some money to balance to proceed: https://vsegpt.ru/User/Money', 'code': 400}}

In [60]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [46]:
print (graph_documents)

In [62]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!INCLUDES]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

display(showGraph())

GraphWidget(layout=Layout(height='800px', width='100%'))

## Hybrid Retrieval for RAG

![retrieval](https://raw.githubusercontent.com/tomasonjo/blogs/master/graphhybrid.png)



In [65]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(openai_api_base="https://api.vsegpt.ru:6070/v1"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

The vector index can then be called with the similarity_search method.
## Graph retriever

![graph](https://raw.githubusercontent.com/tomasonjo/blogs/master/neighbor.png)


In [66]:
# Retriever

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [67]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

The `structured_retriever` function starts by detecting entities in the user question. Next, it iterates over the detected entities and uses a Cypher template to retrieve the neighborhood of relevant nodes. Let's test it out!

In [68]:
print(structured_retriever("Какие бывают скидки?"))

Акция «Бесплатные Ночи» - ПРОДАЖА -> Форма Бронирования
Средство Размещения - ПОДКЛЮЧЕНИЕ -> Тип Тарифа «Бесплатные Ночи»
Средство Размещения - ПОДКЛЮЧЕНИЕ -> Личный Кабинет
Средство Размещения - ПОДКЛЮЧЕНИЕ -> Шахматка Tl: Webpms
Средство Размещения - ИНТЕГРАЦИЯ -> Асу
Раздел Меню «Управление Номерами» - СОЗДАНИЕ -> Тарифы
Free Night Offer - APPLIES_TO -> Guest
Discount Calculation - APPLIES_TO -> Independent Rate
Guest - UPLOAD_ICON -> Icon
Guest - ENTER_PROMO_CODE -> Promo Code
Тариф - СОДЕРЖИТ -> Условия Акции
Special Offer - VISUALIZATION_SETTINGS -> Booking Module
Special Offer - PRICE_DISPLAY_SETTINGS -> Price
Special Offer - MOTIVATOR_SETTINGS -> Price
Special Offer - OFFER_DURATION -> Special Offer
Special Offer - STAY_CONDITIONS -> Special Offer
Special Offer - PAYMENT_OPTIONS -> Payment Methods
Special Offer - SALES_SOURCES -> Sales Channels
Special Offer - CANCELLATION_POLICY -> Cancellation Rule
Special Offer - SPECIAL_NIGHTS -> Nights
Discount - APPLIES_TO -> Last Night
D

## Final retriever
As we mentioned at the start, we'll combine the unstructured and graph retriever to create the final context that will be passed to an LLM.

In [70]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

As we are dealing with Python, we can simply concatenate the outputs using the f-string.
## Defining the RAG chain
We have successfully implemented the retrieval component of the RAG. First, we will introduce the query rewriting part that allows conversational follow up questions.


In [78]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language. 
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0,openai_api_base="https://api.vsegpt.ru:6070/v1")
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

Next, we introduce a prompt that leverages the context provided by the integrated hybrid retriever to produce the response, completing the implementation of the RAG chain.

In [79]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise. Give as detailed instructions as possible to solve the problem.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

Finally, we can go ahead and test our hybrid RAG implementation.

In [73]:
chain.invoke({"question": "Какие бывают скидки?"})

Search query: Какие бывают скидки?


'Скидки могут быть на последнюю ночь, на самостоятельный тариф, на рабочие и выходные дни, на все категории сразу, а также через промокоды.'

Let's test a follow up question!

In [80]:
chain.invoke(
    {
        "question": "Как автоматически отправлять feedback письма?",
        "chat_history": [("Какие бывают скидки?", "Скидки могут быть на последнюю ночь, на самостоятельный тариф, на рабочие и выходные дни, на все категории сразу, а также через промокоды.")],
    }
)

Search query: Как автоматически отправлять feedback письма?


'Для автоматической отправки feedback-писем с промокодами, следуйте инструкциям: \n1. Перейдите в раздел "Маркетинг" > "Email-Рассылка".\n2. Выберите тип письма "Feedback-письмо".\n3. Добавьте промокод в поле "Текст приветствия" и сохраните изменения.\n4. Для отправки письма о незавершенном бронировании с промокодом, выберите тип письма "Незавершенное бронирование для гостя" и добавьте промокод.\n5. Следуйте инструкциям по настройке отправки писем в соответствии с вашими предпочтениями.'